In [3]:
CONSUMER_KEY = 'HMnNDIlLnpxqoyAQC9gSs1ZxX'
CONSUMER_SECRET = 'uba7NCMbtGiLUOSs3cPp2TlzuveBZhUlCkBLLhQiHJFqx9zmvf'
ACCESS_TOKEN = '1153613267195039744-7ZDSA7vWYGJJnylob0or1tj6WHmoN0'
ACCESS_TOKEN_SECRET = 'xGpjqcAcEwzpCvG57DWghb4FwNfjV81UYuavTmxVZiVOb'
HOST = "127.0.0.1"  # Get local machine
PORT = 9999  # Reserve a port for your service.


In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import logging
import threading

In [5]:
LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
             -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
             -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

In [6]:

# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator:

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth


# # # # TWITTER STREAMER # # # #
class TwitterStreamer:
    """
    Class for streaming and processing live tweets.
    """

    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()

    def stream_tweets(self, hash_tag_list, c_socket):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(c_socket)
        auth = self.twitter_autenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords:
        # stream.filter(track=hash_tag_list)
        stream.filter(locations=LOCATIONS, languages=["en"])


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """

    def __init__(self, c_socket):

        self.client_socket = c_socket

    def on_data(self, data):
        try:
            msg = json.loads(data)
            print(msg['text'].encode('utf-8'))
            self.client_socket.send(msg['text'].encode('utf-8'))
            self.client_socket.send("\n".encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True


# create connection
def sendData(c_socket, hash_tag_list):
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(hash_tag_list, c_socket)


def tread_function(port):
    s = socket.socket()  # Create a socket object
    host = twitter_credentials.HOST

    # Reserve a port for your service.
    s.bind((host, port))  # Bind to the port

    hash_tag_list = []
    #hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders", "movie"]

    s.listen(100)  # Now wait for client connection.
    c, addr = s.accept()  # Establish connection with client.

    print("Received request from: " + str(addr))

    sendData(c, hash_tag_list)

In [ ]:
x1 = threading.Thread(target=tread_function, args=(9999,))
x1.start()